# II. Opening a Sentinel-2 image with rasterio

---
**Author(s):** Quentin Yeche, Kenji Ose, Dino Ienco - [UMR TETIS](https://umr-tetis.fr) / [INRAE](https://www.inrae.fr/)

---

## 1. Introduction

In this notebook we will cover basic usage of Python's `rasterio` library in order to download and manipulate satellite imagery. In a first time, we will handle single-band and single-date imagery. Then, we will handle multi-band images and discover color composites.

## 2. Import libraries

As usual, we import all the required Python libraries. Th new one is `rasterio`, a package for accessing the many different kind of raster data files used in the GIS field.

In [ ]:
# STAC access
import pystac_client
import planetary_computer

# dataframes and their geospatial data counterpart
import geopandas as gpd
import pandas as pd

# library for handling GIS rasters
import rasterio as rio
import rasterio.mask

# library for vector data
import shapely
from shapely.geometry import shape

# library for tranforming coordinates
from pyproj import Transformer

#visualization
import folium #maps
from matplotlib import pyplot as plt

# NumPy arrays
import numpy as np

# miscellanous
from IPython.display import display
import json
import rich

## 3. Obtaining an Asset and looking at its properties

### 3.1. Opening landcover data and Sentinel-2 image

Let's reuse the example from the [STAC notebook](./Joensuu_01-STAC.ipynb). We have a GeoJSON of some polygons that make up our area of interest. We have also identified the ID of the STAC `Item` we want to use (it of course encompasses our area of interest).

In [ ]:
# polygons defining our area of interest
with open("./sample.geojson") as file:
    geojson_feature = json.load(file)

# retrieving the relevant STAC Item
catalog = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1",
    modifier=planetary_computer.sign_inplace,
    )

item = catalog.get_collection('sentinel-2-l2a').get_item("S2A_MSIL2A_20201213T104441_R008_T31TEJ_20201214T083443")
item

### 3.2. Sentinel-2 Item's assets

#### 3.2.1. Listing assets of Sentinel-2 image

First, let's have a look at the list of assets available:

In [ ]:
assets = [asset.title for asset in item.assets.values()]
descriptions = pd.DataFrame(assets, columns=['Description'], index=pd.Series(item.assets.keys(), name='asset_key'))
descriptions

With this table, we can easily find the IDs (*asset_key*) of the assets we are looking for. 

#### 3.2.2. Basic description of a Sentinel-2 spectral band

Let's choose the blue band ('B02'). Rather than directly loading the data, we will only access the `profile` property for now.

In [ ]:
with rio.open(item.assets['B02'].href) as ds:
    profile = ds.profile
rich.print(profile)

The `profile` property has two fields we will use shortly. 

The first is `crs` or Coordinate Reference System, which specifies the geospatial coordinate system that the data uses. In this case the CRS is EPSG:32631. As we have seen in the [previous notebook](./Joensuu_01-STAC.ipynb), the polygons defining our area of interest are GPS coordinates, i.e. they use the EPSG:4326 CRS. Thus we have a different coordinate systems for our satellite image and our polygons. This means we will have to convert one into the other before we can work with both. It is usually recommended to preserve the CRS of the imagery (in order to avoid pixel value tranformation with resampling methods). In our case, we will convert the coordinates of our polygons into EPSG:32631.

The second field is `transform`. It describes how to relate geospatial coordinates and pixels. Here's a description of the six values given:
 ```
[0] cos(θ) * x_resolution
[1] -sin(θ) * x_resolution
[2] x-coordinate of upper-left raster corner
[3] sin(θ) * y_resolution
[4] cos(θ) * y_resolution
[5] y-coordinate of upper-left raster corner
 ```

Here (and in most situations) the affine transformation includes no rotation. Since $θ=0$ the previous list can be re-written as:
```
[0] x_resolution
[1] 0
[2] x-coordinate of upper-left raster corner
[3] 0
[4] y_resolution
[5] y-coordinate of upper-left raster corner
```
Thus when reading the `transform` field we can deduce the following:
1. Each pixel has a width corresponding to 10 units in the CRS. For our CRS the units are meters, so a pixel width of 10 meters.
2. The upper-left corner of the satellite image corresponds to a point with geospatial coordinates (499980.0,4900020.0)
3. Each pixel has a height corresponding to -10 meters.

Note: A negative pixel height may seem confusing. But it simply captures the following dichotomy:
1. In an image stored as data, it is standard for the origin to be placed at the upper-left corner. Pixel coordinates increase as you go right, and down.
2. In most CRS (and indeed it's the case for EPSG:32631), coordinate values increase as you go east, and **north**


## 4. Getting the relevant part of satellite image

The reason we retrieve the CRS and transformation matrix before accessing the data itself is a matter of efficiency. We are only interested in a certain area of interest (defined by the GeoJSON file). `rasterio` and the file format of the data (Cloud Optimized GeoTiff or COG) allow us to only download the relevant part of an image, rather than downloading the full image and then cropping it.

As a test you can try downloading the full band by uncommenting and running the following cell. A typical size of a Sentinel-2 is in the hundreds of megabytes. In most projects it is not uncommon to use multiple bands and possibly multiple dates. So reducing the amount of data we need to download is alway a good practice, especially to help with download times.

In [ ]:
"""
with rio.open(item.assets['B02'].href) as ds:
    data = ds.read()
    print(data.nbytes)
    print(f"Total size of the data is {data.nbytes//(2**20)} MB")
"""
None #only there to prevent the string to be output by Jupyter's cell magic

We will see two approaches to obtaining the relevant data: using a window for the extent, and using masks.

### 4.1. Using a window with `rasterio`

If we use a window, we will obtain the reflectance data for the whole extent. This has the advantage of giving us context around each of our polygons. Among others, it can be useful to quickly check whether the area is cloudy (although we will see better methods for that later).

#### 4.1.1. Creating a bounding box
First let's create a bounding box from all the polygons, and convert the bounds to EPSG:32631 using `rasterio.warp.transform_bounds`:

In [ ]:
multipolygon = shapely.geometry.MultiPolygon([shape(feature['geometry']) for feature in geojson_feature['features']])
aoi_bounds = multipolygon.bounds
print(f"bounds in EPSG:4326 {aoi_bounds}")
converted_aoi_bounds = rio.warp.transform_bounds("epsg:4326", profile['crs'], *aoi_bounds)
print(f"bounds after conversion to EPSG:32631 {converted_aoi_bounds}")

With these bounds we can create a `Window` object. We need to specify the affine transform of our data.

In [ ]:
aoi_window = rio.windows.from_bounds(*converted_aoi_bounds, transform = profile['transform'])
aoi_window

We can see in the output of the last cell that the Window object describes the offsets and length of pixel coordinates on the satellite image, so the affine transformation has been applied. The values are floating point numbers instead of integers, but they will automatically be rounded later on.

#### 4.1.2. Reading the data

At this point we are ready to read the data for our band, downloading only the window which encompasses our area of interest:

In [ ]:
with rio.open(item.assets['B02'].href) as ds:
    band = ds.read(window=aoi_window)
print(f"Type: {type(band)} \nShape: {band.shape}\nTotal size: {band.nbytes//2**20} MB")

The result of the `read` method is a NumPy array. At this point the data has been downloaded, but only the necessary parts for the window we specified. This is reflected in both the shape and size of the array, as well as the time required for execution.

There are two things worthy of note concerning the shape of the NumPy array. The first is that it has three dimensions: (channels, height, width). This even applies in our case where there is a single band, so a single channel. This is mostly a matter of consistency, as most geospatial library do not generally differentiate between single-band and multi-band data, hence they always expect a 3-dimensional array. The second important point is that the order of dimensions does not respect the typical (height, width, channels) convention used for images.

A result of those points is that when we want to manipulate the data with general-purpose tools we may have to re-order the dimensions with `transpose` and/or remove the extra channel dimension of size 1 using the `squeeze` method.

In [ ]:
#here we use the squeeze method to remove axes of length one
print(band.squeeze().shape)
plt.imshow(band.squeeze(), cmap='RdGy')

### 4.2. Using masks with `rasterio`

By using masks, we can choose to only keep the reflectance values inside our polygons that define our area of interest. This means that adjusting the contrast can be more relevant as we are not calculating statistics on the whole extent. However we would need to account for the fact that any pixel outside our area of interest does still have a value of 0.

#### 4.2.1. Converting the MultiPolygon into another CRS

In [ ]:
transform = Transformer.from_crs('epsg:4326', 'epsg:32631', always_xy=True).transform
multipolygon_warped = shapely.ops.transform(transform, multipolygon)

#### 4.2.2. Reading the data

In [ ]:
with rio.open(item.assets['B02'].href) as ds:
    out_image, out_transform = rio.mask.mask(ds, [multipolygon_warped], crop=True)

print(out_image.squeeze().shape)
plt.imshow(out_image.squeeze(), cmap='RdGy')#'gray')

## 5. Adjusting the contrast for visuals

### 5.1. Distribution of pixel values of the image

With `matplotlib` we can visualize the image. However it looks very dark. This is quite typical for satellite imagery. A minority of pixels will record very high reflectance values, and default normalization will bring down the contrast on the rest of the pixels. This is easily illustrated with a quick histogram on the pixel values of the image clipped with the `rasterio` window:


In [ ]:
plt.hist(band.reshape(-1), bins=30)
plt.show()
print(f"Max value: {band.max()}")
print(f'{np.count_nonzero(band > 2500)/(band.size):.1%} of values higher than 2500')

### 5.2. Image contrast

We can see that the majority of values fall between 0 and 2000. For producing visuals it is thus helpful to set a cutoff point and clip the values above it. Here we arbitrarily choose 2500 but some standards practices include using  percentiles (usually [2-98] or [5-95]) or defining intervals based on the standard deviation of the data. Here's one way to clip the values:

In [ ]:
# Getting min and max with Percentiles [2-98] method
m1_min = (np.percentile(band, 2))
m1_max = (np.percentile(band, 98))
print(f"Percentiles method: min={m1_min} ; max={m1_max}")

# Getting min and max with Standard deviation method
m2_mean = band.mean()
m2_stdv = band.std()
print(f"Percentiles method: min={int(band.mean()-band.std())} ; max={int(band.mean()+band.std())}")

Now, we plot the Sentinel-2 band, before and after values stretching.

In [ ]:
band_visual = band.squeeze().copy()
band_visual[band_visual>2500] = 2500

f = plt.figure()
f.add_subplot(1,2, 1)
plt.imshow(band.squeeze(), cmap='gray')
plt.title('before')
f.add_subplot(1,2, 2)
plt.imshow(band_visual, cmap='gray')
plt.title('after')
plt.show(block=True)

However `imshow` already provides us with tools to specify some cutoff point: `vmax`. Any value higher than this value will be clipped down to that cutoff.

**Note:** There also exists `vmin` to specify a lower bound cutoff but since reflectance values are often grouped near zero this cutoff is typically less useful.

In [ ]:
plt.imshow(band.squeeze(), vmax=2500, cmap='gray')

> **Exercise**: Plot the Sentinel-2 band with percentiles and standard deviation methods.

In [ ]:
# to fill
# ...
# ...

### 5.3 Visualization on a map

Let's now visualize the band over a map:

In [ ]:
# once again, standards for bounding box coordinates can differ
b = [[aoi_bounds[1], aoi_bounds[0]], [aoi_bounds[3], aoi_bounds[2]]]
maps = folium.Map(tiles=None, location=[43.6085, 4.0053], zoom_start=11, control_scale=True)
folium.TileLayer("http://mt0.google.com/vt/lyrs=s&hl=en&x={x}&y={y}&z={z}", attr='Google', name='Google Satellite').add_to(maps)
folium.TileLayer(name='OpenStreetMap').add_to(maps)
folium.raster_layers.ImageOverlay(out_image.squeeze(), bounds=b,overlay=True, cross_origin=False, name="B02_mask").add_to(maps)
folium.raster_layers.ImageOverlay(band_visual, bounds=b,overlay=True, cross_origin=False, name="B02_window").add_to(maps)
folium.LayerControl().add_to(maps)

maps

## 6. Management of color composites

We will now load several spectral bands and projet them through the RGB color space in order to produce colored images, or color composites. It is based on the principale of additive color:

<figure align="center">
  <img src="https://upload.wikimedia.org/wikipedia/commons/thumb/e/e0/Synthese%2B.svg/1024px-Synthese%2B.svg.png" width="30%" alt="additive color">
  <figcaption>STAC Collection Specification </br> Figure by <a href="https://commons.wikimedia.org/w/index.php?curid=818982">Wikipedia</a> (license <a href="https://creativecommons.org/licenses/by/3.0">CC BY-SA 3.0</a>)
  </figcaption>
</figure>



In the world of remote sensing, there are two well-known color composites:

- the **natural colors composite**, or true colors, consists in assigning respectively to the red, green and blue filters the red, green and blue spectral bands. On display, the color reproduction is very close to what a human could perceive from space. Vegetation appears in shades of green, water in blue, urban areas in light gray.
- the **false-color composite** images display spectral bands which are different from primary colors. In fact, the color of the objects does not correspond to their “real” color as our eye would perceive it. The false-color is used to highlight certain objects in the image, taking advantage of their spectral signature. Therefore, the composition in infrared false colors is commonly used to study vegetation. It is generally accepted that the infrared, red and green spectral bands are allocated, respectively, to the red, green and blue filters. Vegetation is seen in red since its reflectance is at its maximum in the infrared. The shades vary according to the chlorophyll activity of the species in the environment.

### 6.1. Loading 10m Sentinel-2 spectral bands

Here, we will produce color composites based on Sentinel-2 10m spectral bands: 'B02', 'B03','B04' and 'B08'. In the first time, we have to load each asset.

In [ ]:
with rio.open(item.assets['B02'].href) as ds:
    band_blue = ds.read(window=aoi_window)
with rio.open(item.assets['B03'].href) as ds:
    band_green = ds.read(window=aoi_window)
with rio.open(item.assets['B04'].href) as ds:
    band_red = ds.read(window=aoi_window)
with rio.open(item.assets['B08'].href) as ds:
    band_nir = ds.read(window=aoi_window)

### 6.2. Normalize band values

In order to display RGB image with `matplotlib`, we need to normalize each band between \[0, 1\]. To do this, we propose to define a function called `normalize`. It takes two arguments:
- array: numpy ndarray for each band
- vmax: max pixel value in order to improve the contrast of the output band

In [ ]:
# Function to normalize the grid values
def normalize(array, vmax=False):
    """Normalizes numpy arrays into scale 0.0 - 1.0"""
    if vmax:
        array[array>vmax] = vmax
    array_min, array_max = array.min(), array.max()
    return ((array - array_min)/(array_max - array_min))

band_blue_n = normalize(band_blue, 2500)
band_green_n = normalize(band_green, 2500)
band_red_n = normalize(band_red, 2500)
band_nir_n = normalize(band_nir, 2500)

### 6.3. Bands stacking and display

We stack all the bands to produce a natural color composite.

In [ ]:
rgb = np.stack((band_red_n, band_green_n, band_blue_n))
rgb_natcol = np.moveaxis(rgb.squeeze(), 0, -1)

Finally, we can plot the RGB image.

In [ ]:
plt.imshow(rgb_natcol)

> **Exercise**: Display a infrared false color composite.


In [ ]:
# to fill
# ...
# ...

> **Question**:
> - Why vegetation appears in red?


.........................................
.........................................
.........................................

> **Question**:
    - Could you propose a infrared false color composite where vegetation appears in green?

.........................................
.........................................
.........................................